In [ ]:
import tensorflow as tf
from flow_models import *
from model_trainer_half import AnimeModel
from tensorflow.keras import mixed_precision
import numpy as np
from pathlib import Path
import cv2
from tools.stitch import *

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

In [ ]:
patch_size = (320,320)
overlap = 60
interp_ratio = [0.4,0.8]
model_f = ehrb0_143_32
weight_dir = 'savedmodels/ehrb0_500/66'

In [ ]:
inputs = tf.keras.Input((patch_size[1],patch_size[0],6))
anime_model = AnimeModel(inputs, model_f, interp_ratio)
anime_model.load_weights(weight_dir)

In [ ]:
vid_dir = Path('interp/to_convert')
vid_paths = [str(vid_dir/vn) for vn in os.listdir(vid_dir)]

In [ ]:
from tqdm.notebook import trange

In [ ]:
cap = cv2.VideoCapture(vid_paths[0])
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
writer = None
ret, frame = cap.read()
frame_size = (frame.shape[1],frame.shape[0])
frame_size_hw = (frame_size[1],frame_size[0])
writer = cv2.VideoWriter(f'{vid_paths[0]}_interp.mp4',fourcc,60,frame_size)
for i in trange(1600):
    if not cap.isOpened():
        break
    if ret:
        frame0 = frame
    else:
        break
        
    
    ret, frame = cap.read()
    if ret:
        frame1 = frame
    else:
        break

    ret, frame = cap.read()
    if ret:
        frame2 = frame
    else:
        break

    concated1 = np.concatenate([frame0,frame1],axis=-1).astype(np.float32)/ 255.0
    concated2 = np.concatenate([frame2,frame1],axis=-1).astype(np.float32)/ 255.0
    patches = frame_to_patch_on_batch(np.array([concated1,concated2]),patch_size,overlap)
    outputs = anime_model.predict_on_batch(patches)
    outputs = np.round(np.clip(outputs, 0, 1) * 255).astype(np.uint8)
    interped = patch_to_frame_on_batch(outputs,frame_size_hw,overlap)
    interped1, interped2 = interped[0][...,0:3], interped[0][...,3:6]
    interped3, interped4 = interped[1][...,3:6], interped[1][...,0:3]
    writer.write(frame0)
    writer.write(interped1)
    writer.write(interped2)
    writer.write(interped3)
    writer.write(interped4)


cap.release()
writer.release()